<a href="https://colab.research.google.com/github/jyonalee/Insider-Threat-and-Anomaly-Detection-from-User-Activities/blob/master/Anomaly_Detection_LSTM_Data_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install awscli to download the data
!pip3 install awscli --upgrade --user

# download data and save it on `data`
!mkdir data
!~/.local/bin/aws s3 sync --no-sign-request --region us-west-1 "s3://cse-cic-ids2018/Processed Traffic Data for ML Algorithms/" data/.

    100% |████████████████████████████████| 1.5MB 10.8MB/s 
    100% |████████████████████████████████| 5.3MB 5.0MB/s 
    100% |████████████████████████████████| 51kB 18.8MB/s 
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
  The scripts pyrsa-decrypt, pyrsa-decrypt-bigfile, pyrsa-encrypt, pyrsa-encrypt-bigfile, pyrsa-keygen, pyrsa-priv2pub, pyrsa-sign and pyrsa-verify are installed in '/root/.local/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


download: s3://cse-cic-ids2018/Processed Traffic Data for ML Algorithms/Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv to data/Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv
download: s3://cse-cic-ids2018/Processed Traffic Data for ML Algorithms/Friday-02-03-2018_TrafficForML_CICFlowMeter.csv to data/Friday-02-03-2018_TrafficForML_CICFlowMeter.csv
download: s3://cse-cic-ids2018/Processed Traffic Data for ML Algorithms/Friday-16-02-2018_TrafficForML_CICFlowMeter.csv to data/Friday-16-02-2018_TrafficForML_CICFlowMeter.csv
download: s3://cse-cic-ids2018/Processed Traffic Data for ML Algorithms/Friday-23-02-2018_TrafficForML_CICFlowMeter.csv to data/Friday-23-02-2018_TrafficForML_CICFlowMeter.csv
download: s3://cse-cic-ids2018/Processed Traffic Data for ML Algorithms/Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv to data/Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv
download: s3://cse-cic-ids2018/Processed Traffic Data for ML Algorithms/Thursday-22-02-2018_TrafficForML_CICFl

# Anomaly Detection with LSTM in Network Traffic Data

This project explores anomaly detection in network traffic with RNN-LSTM to train the model.

The dataset can be obtained [here](https://www.unb.ca/cic/datasets/ids-2018.html)

This is part of the capstone project for the Machine Learning Nano Degree from Udacity

##Data Exploration

---



In [0]:
import pandas as pd
import numpy as np
import os
import glob

In [3]:
# load data and do preliminary cleaning

filepath = '/content/data/Thursday-01-03-2018_TrafficForML_CICFlowMeter.csv'
df1 = pd.read_csv(filepath)
df1 = df1[df1['Protocol'] != 'Protocol']
filepath = '/content/data/Friday-16-02-2018_TrafficForML_CICFlowMeter.csv'
df2 = pd.read_csv(filepath)
df2 = df2[df2['Protocol'] != 'Protocol']
filepath = '/content/data/Friday-02-03-2018_TrafficForML_CICFlowMeter.csv'
df3 = pd.read_csv(filepath)
filepath = '/content/data/Friday-23-02-2018_TrafficForML_CICFlowMeter.csv'
df4 = pd.read_csv(filepath)
filepath = '/content/data/Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv'
df5 = pd.read_csv(filepath)
filepath = '/content/data/Wednesday-14-02-2018_TrafficForML_CICFlowMeter.csv'
df6 = pd.read_csv(filepath)
filepath = '/content/data/Thursday-22-02-2018_TrafficForML_CICFlowMeter.csv'
df7 = pd.read_csv(filepath)
filepath = '/content/data/Wednesday-28-02-2018_TrafficForML_CICFlowMeter.csv'
df8 = pd.read_csv(filepath)
df8 = df8[df8['Protocol'] != 'Protocol']
filepath = '/content/data/Wednesday-21-02-2018_TrafficForML_CICFlowMeter.csv'
df9 = pd.read_csv(filepath)


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py

In [4]:
# df9.head()
# len(df1)
print(len(df1)+len(df2)+len(df3)+len(df4)+len(df5)+len(df6)+len(df7)+len(df8)+len(df9))


8284195


In [0]:
df = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9], ignore_index=True)

In [0]:
# clean up intermediary dataframes to free memory
del df1
del df2
del df3
del df4
del df5
del df6
del df7
del df8
del df9